In [1]:
# imports
from pathlib import Path
import numpy as np
from scipy.signal import resample
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout, BatchNormalization, MaxPooling1D
from keras import regularizers
from keras import initializers
from keras.optimizers import Adam, SGD, Adagrad
from keras.utils import np_utils
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
## data loading
datapath = Path('../dataset')
xfile = 'X_features_spec.npy'
yfile = 'Y_labels_spec.npy'
def load_waveforms():
    X_list = np.load(str(datapath.joinpath(xfile)))
    Y_list = np.load(str(datapath.joinpath(yfile)))
    return X_list, Y_list

def positve_samples(xlist):
    ## some samples have negative signs
    xl_new = []
    for sample in range(xlist.shape[0]):
        points = xlist[sample]
        for p in range(points.shape[0]):
            point = points[p]
            if np.sum(point) < 0:
                points[p] = -point
        xl_new.append(points)
    return np.array(xl_new)

## apply to loaded dataset
def split_by_channel(xlist):
    ## input as (n, 2500)
    def standard_resample(arr):
        return resample(arr, 2500)
    ## if some is not with dim 625, resample it
    xl_new = []
    for sample in range(xlist.shape[0]):
        points = xlist[sample]
        if points.shape[1] != 2500:
            print("resample")
            print(points.shape)
            points = np.apply_along_axis(standard_resample, axis=1, arr=points)
        points = points.reshape((points.shape[0], 625, 4))
        xl_new.append(points)
    return np.array(xl_new)

## input is after split
def apply_resample(xlist, outdim):
    ## resample
    def resample_waveform(arr):
        ## arr.shape: (indim, )
        return resample(arr, outdim)
    xl_new = []
    for sample in range(xlist.shape[0]):
        points = xlist[sample]
        points = np.apply_along_axis(resample_waveform, axis=1, arr=points)
        xl_new.append(points)
    return np.array(xl_new)

## input is combined exp. (18000 ,625, 4)
def get_xtrain_mean(x_train):
    ## mean value for each dimension (exp. each of 625 dim)
    m = np.mean(x_train, axis=0)
    ## then we can apply x_train - m for zero mean
    return m

## input is after split
## one variance for each channel
def normalize_waveform():
    ## we don't necessarily need this
    pass

def combine_samples(arrs):
    ## exp. arrs.shape: (20, ?)
    pass

def binary_label(ylist):
    ## 1, 2 --> 1
    ylist_new = []
    for sample in range(ylist.shape[0]):
        labels = ylist[sample]
        labels[labels > 1] = 1
        ylist_new.append(labels)
    return np.array(ylist_new)

def combine_samples(arrs):
    ## exp. arrs.shape: (20, ?)
    if arrs.shape[0] < 1:
        return arrs
    sp = list(arrs[0].shape)
    sp[0] = 0
    combined = np.zeros(sp)
    print("combinde", combined.shape)
    for sample in range(arrs.shape[0]):
        arr = arrs[sample]
        combined = np.concatenate((combined, arr), axis=0)
    return combined

In [3]:
## global parameters

### models to test
* VERY DEEP CONVOLUTIONAL NEURAL NETWORKS FOR RAW WAVEFORMS

they have a clearly defined structure, and their data are of similar dimentions

* Raw Waveform-based Audio Classification Using Sample-level CNN Architectures
* SAMPLE-LEVEL DEEP CONVOLUTIONAL NEURAL NETWORKS FOR MUSIC AUTO-TAGGING USING RAW WAVEFORMS

realtively simple arch;

#### test raw waveform input first
* input: 2500 * 1 waveform (normalized, center to 0, variance 1)
* conv layer: with/withour overlapping. In the paper:
    * filter size 3, stride 3, 128 filters
    * filter size 80, stride 4, 256 filters
* batch normalization: after every conv layer
* max pool
    * stride of 3? 4?


### input
or we can make the input as 625 * 4

In [4]:
## model
## 1d conv, size 4 filter, 64 filters, stride 2
## output 1250 * 64
## batch norm
## maxpool 2 * 1
## output 625 * 64
## 1d conv, size 3 filter, stride 2, 128 filters
## maxpool 2 * 1
## output 312 * 64
## 1d conv, size 3 filter, stride 3, 128 filters
## output 104 * 128
## maxpool 2 * 1
## output 52 * 128
## 1d conv, size 3 filter, stride 2, 256 filters

In [17]:
## one time parameter for the model below
## regularizer
## l2
ker_reg = 0.1
act_reg = 0.1
## kernel_initializer
ker_init = initializers.glorot_normal(seed=None)
## shape
in_shape = (648, 4)
## learning rate
opt = Adam()
opt.lr = 0.0001
##
OUTPUT_SIZE = 2
## batch size
bsize = 50
##
epochs = 30
## callback
model_callback = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)

In [18]:
## model
## resample data to 648 * 1
model = Sequential()
## 1d conv, size 3 filter, 64 filters, stride 1
## batch norm, batch after activation
## no maxpool
## keras.layers.Conv1D(filters, kernel_size, strides=1, padding='valid', dilation_rate=1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
model.add(Convolution1D(filters=64, kernel_size=3, strides=1, padding='same', input_shape=in_shape, kernel_initializer=ker_init, activation='relu', kernel_regularizer=regularizers.l2(ker_reg)))
model.add(BatchNormalization())
## 1d conv, size 3 filter, 128 filters, stride 1
## batch norm, batch after activation
## maxpool 3 --> 216 * 128
model.add(Convolution1D(filters=128, kernel_size=3, strides=1, padding='same', input_shape=in_shape, activation='relu', kernel_regularizer=regularizers.l2(ker_reg)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=3))
## 1d conv, size 3 filter, 128 filters, stride 2
## batch norm, batch after activation
## max pool 3 -->  36 * 128
model.add(Convolution1D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu', kernel_regularizer=regularizers.l2(ker_reg)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=3))
## 1d conv, size 3 filter, 256 filters, stride 2
## batch norm, batch after activation
## max pool 3 -->  6 * 256
model.add(Convolution1D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu', kernel_regularizer=regularizers.l2(ker_reg)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=3))
## 1d conv, size 3 filter, 512 filters, stride 2
## batch norm, batch after activation
## max pool 3 -->  1 * 512
model.add(Convolution1D(filters=512, kernel_size=3, strides=2, padding='same',activation='relu', kernel_regularizer=regularizers.l2(ker_reg)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=3))
##
model.add(Flatten())
## fully connected
model.add(Dense(OUTPUT_SIZE))
## softmax
model.add(Activation('softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_11 (Conv1D)           (None, 648, 64)           832       
_________________________________________________________________
batch_normalization_11 (Batc (None, 648, 64)           256       
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 648, 128)          24704     
_________________________________________________________________
batch_normalization_12 (Batc (None, 648, 128)          512       
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 216, 128)          0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 108, 128)          49280     
_________________________________________________________________
batch_normalization_13 (Batc (None, 108, 128)          512       
__________

In [8]:
## experiment
## case 12, 14 for test
## case 18 for validation
## other case for traning
x_list, y_list = load_waveforms()
x_list = positve_samples(x_list)
x_list = split_by_channel(x_list)
x_list = apply_resample(x_list, 648)
y_list = binary_label(y_list)
for i in range(x_list.shape[0]):
    print(x_list[i].shape)

resample
(2490, 1920)
resample
(878, 1920)
(2394, 648, 4)
(4144, 648, 4)
(3302, 648, 4)
(1272, 648, 4)
(389, 648, 4)
(2716, 648, 4)
(61, 648, 4)
(628, 648, 4)
(611, 648, 4)
(771, 648, 4)
(201, 648, 4)
(1354, 648, 4)
(2490, 648, 4)
(878, 648, 4)
(2506, 648, 4)
(1688, 648, 4)
(2067, 648, 4)
(1554, 648, 4)
(635, 648, 4)
(1439, 648, 4)


In [9]:
val_idx = [17]
test_idx = [11, 13]
train_list_x = []
train_list_y = []
val_list_x = []
val_list_y = []
test_list_x = []
test_list_y = []
for idx in range(x_list.shape[0]):
    if idx not in (val_idx + test_idx):
        train_list_x.append(x_list[idx])
        train_list_y.append(y_list[idx])
        
for idx in val_idx:
    val_list_x.append(x_list[idx])
    val_list_y.append(y_list[idx])
    
for idx in test_idx:
    test_list_x.append(x_list[idx])
    test_list_y.append(y_list[idx])  

train_list_x = np.array(train_list_x)
train_list_y = np.array(train_list_y)
val_list_x = np.array(val_list_x)
val_list_y = np.array(val_list_y)
test_list_x = np.array(test_list_x)
test_list_y = np.array(test_list_y)
train_list_x = combine_samples(train_list_x)
train_list_y = combine_samples(train_list_y)
val_list_x = combine_samples(val_list_x)
val_list_y = combine_samples(val_list_y)
test_list_x = combine_samples(test_list_x)
test_list_y = combine_samples(test_list_y)
train_list_y = np_utils.to_categorical(train_list_y, num_classes=2)
val_list_y = np_utils.to_categorical(val_list_y, num_classes=2)
test_list_y = np_utils.to_categorical(test_list_y, num_classes=2)

combinde (0, 648, 4)
combinde (0,)
combinde (0, 648, 4)
combinde (0,)
combinde (0, 648, 4)
combinde (0,)


In [20]:
model.fit(train_list_x, train_list_y,
          epochs=epochs,
          batch_size=bsize
          verbose=2,
          validation_data=(test_list_x, test_list_y),
          callbacks=[model_callback])

Train on 27314 samples, validate on 2232 samples
Epoch 1/30
 - 15s - loss: 28.8543 - acc: 0.9401 - val_loss: 9.9680 - val_acc: 0.3342
Epoch 2/30
 - 15s - loss: 4.4028 - acc: 0.9463 - val_loss: 3.4727 - val_acc: 0.4350
Epoch 3/30
 - 15s - loss: 1.2810 - acc: 0.9487 - val_loss: 1.8393 - val_acc: 0.3925
Epoch 4/30
 - 15s - loss: 0.5820 - acc: 0.9499 - val_loss: 1.3819 - val_acc: 0.5506
Epoch 5/30
 - 15s - loss: 0.3583 - acc: 0.9516 - val_loss: 1.0141 - val_acc: 0.6577
Epoch 6/30
 - 15s - loss: 0.2699 - acc: 0.9534 - val_loss: 1.1281 - val_acc: 0.5981
Epoch 7/30
 - 15s - loss: 0.2281 - acc: 0.9535 - val_loss: 1.3551 - val_acc: 0.3678
Epoch 8/30
 - 15s - loss: 0.2077 - acc: 0.9545 - val_loss: 2.0228 - val_acc: 0.2509
Epoch 9/30
 - 15s - loss: 0.1943 - acc: 0.9551 - val_loss: 1.6658 - val_acc: 0.3513
Epoch 10/30
 - 15s - loss: 0.1771 - acc: 0.9582 - val_loss: 1.7969 - val_acc: 0.3297
Epoch 11/30
 - 15s - loss: 0.1704 - acc: 0.9576 - val_loss: 1.7024 - val_acc: 0.3866
Epoch 12/30
 - 15s - los

In [21]:
loss, acc = model.evaluate(test_list_x, test_list_y)
print("loss", loss, "acc", acc)

2232/2232 [==============================] - 0s 191us/step
loss 1.622199524047127 acc 0.3333333333333333
